In [1]:
#import library
import cv2
!pip install ultralytics
import ultralytics
ultralytics.checks()

!pip install supervision
import supervision as sv

Ultralytics YOLOv8.2.75 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.6/78.2 GB disk)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 8.0 MB/s eta 0:00:00


In [2]:
#load pretrained yolov8
from ultralytics import YOLO
model = YOLO("yolov8x.pt")
model.fuse()

100%|██████████| 131M/131M [00:01<00:00, 95.6MB/s]


YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


In [3]:
#select specefic object for predictions
CLASS_NAMES_DICT = model.model.names
selected_classes = [2,3,5,7]

In [17]:
#define functions to draw bbx around the predictions
def box_anotator(frame, detections):

    for detection_idx in range(len(detections)):
        x1, y1, x2, y2 = detections.xyxy[detection_idx].astype(int)
        car_type = detections.data['class_name'][detection_idx]

        if car_type == 'car' :

              cv2.rectangle(img = frame,pt1 = (x1, y1),pt2 = (x2, y2),color = (255, 255, 0),thickness= 2,)
              # cv2.putText(frame, f'{car_type}',(x1, y1), cv2.FONT_HERSHEY_SIMPLEX ,  1, (255,255,255), 1, cv2.LINE_AA)

        elif car_type == 'truck' :

              cv2.rectangle(img = frame,pt1 = (x1, y1),pt2 = (x2, y2),color = (0, 255, 255),thickness= 2,)
              # cv2.putText(frame, f':{car_type}', (x2, y2), cv2.FONT_HERSHEY_SIMPLEX ,  1, (0,0,0), 1, cv2.LINE_AA)

    return frame

In [23]:
#define functions to calc number of car on the road
def crowd_calculator(frame, detections, height, width, in_car, out_car):

    for detection_idx in range(len(detections)):
        x1, y1, x2, y2 = detections.xyxy[detection_idx].astype(int)

        #calc number of car for left half of picture
        if height < y1 <= height +10 and x1 < width//2 :
           in_car += 1

        #calc number of car for right half of picture
        if height+10 > y1 > height  and x1 > width//2:
           out_car += 1

    #make and plot the line counter
    cv2.line(frame,(0, height), (width, height), (0,0, 255), 2)

    if in_car < 50 and out_car < 50:

        cv2.putText(frame, f'trafic_flow: {in_car}: light traffic', (0,50), cv2.FONT_HERSHEY_SIMPLEX ,  1, (255,0,0), 2, cv2.LINE_AA)
        cv2.putText(frame, f'trafic_flow: {out_car}: light traffic', (0, 100), cv2.FONT_HERSHEY_SIMPLEX ,  1, (255,0,0), 2, cv2.LINE_AA)
    else :
        cv2.putText(frame, f'trafic_flow: {in_car}: dense traffic', (0, 50), cv2.FONT_HERSHEY_SIMPLEX ,  1, (255,0,0), 2, cv2.LINE_AA)
        cv2.putText(frame, f'trafic_flow: {out_car}: dense traffic', (0, 100), cv2.FONT_HERSHEY_SIMPLEX ,  1, (255,0,0), 2, cv2.LINE_AA)

    return frame, in_car, out_car



In [24]:
video_path = '/content/sample_video.mp4'

import numpy as np

byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)
cap = cv2.VideoCapture(video_path)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (1280,  720))

in_car = 0
out_car = 0

while cap.isOpened():

    ret, frame = cap.read()
    # Break the loop if the end of the video is reached
    if not ret:
        break
    #grab info of video frame
    height, width, channels = frame.shape
    results = model(frame, verbose = False)[0]

    # convert to Detections
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above
    detections = detections[np.isin(detections.class_id, selected_classes)]

    detections = byte_tracker.update_with_detections(detections)
    print(detections.data['class_name'])
    frame = box_anotator(frame, detections)

    height  = int(height-(height*1/3))
    width = int(width)

    frame, in_car, out_car = crowd_calculator(frame, detections, height, width, in_car, out_car)

    out.write(frame)

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

SupervisionWarnings: `track_buffer` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'lost_track_buffer' instead.
SupervisionWarnings: `track_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'track_activation_threshold' instead.
SupervisionWarnings: `match_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'minimum_matching_threshold' instead.


['car' 'truck' 'car' 'car' 'truck']
['car' 'car' 'truck' 'car' 'truck']
['car' 'car' 'car' 'truck' 'truck']
['car' 'car' 'truck' 'car' 'truck']
['car' 'truck' 'car' 'car' 'car' 'truck']
['car' 'truck' 'car' 'car' 'car' 'truck']
['truck' 'car' 'car' 'car' 'truck']
['truck' 'car' 'car' 'car' 'truck']
['truck' 'car' 'car' 'car' 'car' 'truck']
['car' 'truck' 'car' 'car' 'car' 'car' 'truck']
['car' 'truck' 'car' 'car' 'car' 'car' 'car']
['car' 'truck' 'car' 'car' 'car' 'car' 'car']
['car' 'truck' 'car' 'car' 'car' 'car']
['car' 'truck' 'car' 'car' 'car' 'car' 'car']
['car' 'truck' 'car' 'car' 'car' 'car' 'car']
['car' 'truck' 'car' 'car' 'car' 'car' 'car']
['car' 'truck' 'car' 'car' 'car' 'car' 'truck']
['car' 'truck' 'car' 'car' 'car' 'car' 'car' 'truck']
['car' 'truck' 'car' 'car' 'car' 'car' 'car']
['car' 'truck' 'car' 'car' 'car' 'car' 'car']
['car' 'truck' 'car' 'car' 'car' 'car' 'car']
['car' 'truck' 'car' 'car' 'car' 'car' 'car']
['car' 'truck' 'car' 'car' 'car' 'car' 'car']
['car' '